## Embedding

### 1. Download sample data

In [ ]:
import requests

def download_file(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

url = 'https://raw.githubusercontent.com/dream-365/SparrowRecSys/master/src/main/resources/webroot/sampledata/ratings.csv'
save_path = 'ratings.csv'

download_file(url, save_path)

### 2. Run embedding in spark

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import Embedding

conf = SparkConf().setAppName('ctrModel').setMaster('local[4]')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

rawSampleDataPath = "ratings.csv"
embLength = 10
samples = Embedding.processItemSequence(spark, rawSampleDataPath)
model = Embedding.trainItem2vec(spark, samples, embLength,
                          embOutputPath="emb/item2vecEmb.csv", saveToRedis=False,
                          redisKeyPrefix="i2vEmb")
Embedding.graphEmb(samples, spark, embLength, embOutputFilename="emb/itemGraphEmb.csv",
             saveToRedis=True, redisKeyPrefix="graphEmb")
Embedding.generateUserEmb(spark, rawSampleDataPath, model, embLength,
                    embOutputPath="emb/userEmb.csv", saveToRedis=False,
                    redisKeyPrefix="uEmb")